In [67]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from fashion import fashion

In [69]:
'''
1. LOADING DATASET
'''

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = fashion(root='./mnist', 
                            train=True, 
                            transform=transform,
                            download=True
                           )

test_dataset = fashion(root='./mnist', 
                            train=False, 
                            transform=transform,
                           )

Processing...
Done!


In [76]:
'''
2. MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3500
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [77]:
'''
STEP 2: MODEL
'''

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out


In [78]:
'''
4. INSTANTIATE MODEL CLASS
'''

model = CNNModel()

In [79]:
'''
5. INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

In [80]:
'''
6. INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.028

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [81]:

'''
7. TRAIN MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Iteration: 500. Loss: 0.2734381854534149. Accuracy: 87.4
Iteration: 1000. Loss: 0.27117589116096497. Accuracy: 88.34
Iteration: 1500. Loss: 0.44118019938468933. Accuracy: 89.93
Iteration: 2000. Loss: 0.2719082832336426. Accuracy: 89.93
Iteration: 2500. Loss: 0.30523139238357544. Accuracy: 89.95
Iteration: 3000. Loss: 0.2853061258792877. Accuracy: 89.71


In [75]:
### Save model
save_model = True
if save_model is True:
    #saves only params
    torch.save(model.state_dict(), 'fashion_model_0.028_learning.pk1')